pagina de referencia: https://datascienceparichay.com/article/jaccard-similarity-python/

In [1]:
import pandas as pd
import json

In [2]:
df = pd.read_csv('tweets_2022_abril_junio.csv') # cuidado al ejecutar

In [3]:
df.drop(['id', 'created_at', 'retweet_count', 'favorite_count'], axis=1, inplace=True)

In [4]:
df.head()

,screen_name,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...


In [5]:
df.shape

(4594980, 2)

In [6]:
max = 0
id = 0
for i in range(df.shape[0]):
    if len(df['text'][i]) >= max:
        max = len(df['text'][i])
        id = i

In [7]:
len(df['text'][id]), id #maximo de caracteres y su id

(203, 888457)

In [8]:
min = len(df['text'][id])
id_2 = 1
for i in range(df.shape[0]):
    if len(df['text'][i]) <= min:
        min = len(df['text'][i])
        id_2 = i

In [9]:
len(df['text'][id_2]), id_2 #minimo de caracteres y su id

(1, 4559771)

In [10]:
def jaccard_similarity(A, B):
    #Find intersection of two sets
    nominator = set(A).intersection(set(B))

    #Find union of two sets
    denominator = set(A).union(set(B))

    #Take the ratio of sizes
    if len(nominator) == 0 or len(denominator) == 0:
        return 0
    similarity = len(nominator)/len(denominator)
    
    return similarity

In [11]:
df['text'][0][:4]

'RT @'

In [12]:
tweets_originales = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == False]

rt = df.loc[df['text'].str[:4].str.contains('RT @',case=False) == True]

In [13]:
tweets_originales.head()

,screen_name,text
6,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
9,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
12,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
13,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
14,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...


In [14]:
rt.head()

,screen_name,text
0,h0l4d4ni3l4,RT @ValeMirandaCC: Tras casi 50 años del golpe...
1,Claudio70932894,RT @UTDTrabajoDigno: Mañana jueves a las 18hrs...
2,Cesar_A_RR,RT @JaimeGuajardoR: Aquí está el aporte de @te...
3,rosmarieher,RT @melnicksergio: la pelotudez no tiene limit...
4,GQuelluen,RT @BSepulvedaHales: Ante la circulación de no...


In [15]:
retweets = pd.DataFrame()

In [16]:
split_text = rt['text'].str.split(' ', expand=True, n=2)
retweets['user'] = split_text[1].str.strip("@").str.strip(':')
retweets['tweet'] = split_text[2]


In [17]:
retweets.head()

,user,tweet
0,ValeMirandaCC,"Tras casi 50 años del golpe, la Constitución s..."
1,UTDTrabajoDigno,Mañana jueves a las 18hrs. comienza nuestro pr...
2,JaimeGuajardoR,Aquí está el aporte de @tere_marinovic con res...
3,melnicksergio,la pelotudez no tiene limites...no tiene
4,BSepulvedaHales,"Ante la circulación de noticias falsas, les qu..."


In [18]:
retweets.reset_index(drop=True, inplace=True)

retweets

,user,tweet
0,ValeMirandaCC,"Tras casi 50 años del golpe, la Constitución s..."
1,UTDTrabajoDigno,Mañana jueves a las 18hrs. comienza nuestro pr...
2,JaimeGuajardoR,Aquí está el aporte de @tere_marinovic con res...
3,melnicksergio,la pelotudez no tiene limites...no tiene
4,BSepulvedaHales,"Ante la circulación de noticias falsas, les qu..."
...,...,...
3327415,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
3327416,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
3327417,Gonz1Gorjeperez,@tere_marinovic https://t.co/OWoyaR8Dch
3327418,Gonz1Gorjeperez,@tere_marinovic https://t.co/XWq7hJ3kyv


In [19]:
tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)

C:\Users\jor_x\AppData\Local\Temp\ipykernel_8200\2716881129.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_originales.rename(columns={'screen_name': 'user', 'text': 'tweet'}, inplace=True)


In [20]:
tweets = pd.concat([tweets_originales, retweets]).reset_index(drop=True)

tweets

,user,tweet
0,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...
4594975,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
4594976,DanielAbelLope1,@tere_marinovic 😡🤮😡🤮 VIEJO C.S.M. 🤮😡🤮 https://...
4594977,Gonz1Gorjeperez,@tere_marinovic https://t.co/OWoyaR8Dch
4594978,Gonz1Gorjeperez,@tere_marinovic https://t.co/XWq7hJ3kyv


In [21]:
final_tweets = tweets.drop_duplicates(['user', 'tweet'], ignore_index=True)

final_tweets

,user,tweet
0,simonlodijo,@unveranonaranja @ruidosafest @franciscamusic ...
1,MacaSimplemente,@LaSuRivas @ElisaLoncon @siliconvalle @Valdebe...
2,LuisVer75699645,@teanval0207 @izkia @arturozunigaj Excelente...
3,MITERRED,@mcubillossigall https://t.co/gkg5PwbZhZ
4,piaelizabethvm,@simonlodijo @ruidosafest @franciscamusic @gio...
...,...,...
1442712,hocikonapatriot,@tere_marinovic fuiste la voz de muchos chilen...
1442713,Caroline_walls6,@tere_marinovic Si la Tere pudiera decir lo qu...
1442714,EduardoAlegri16,@tere_marinovic Justamente la mayor ausente e...
1442715,AlonsoRPatriota,"@maluchayallego Historica debería ser, el dar ..."


In [22]:
tweet_shingles = dict()
inverted_index = dict()
k=5

for id in range(1000): 
    user = final_tweets.iloc[id]['user']
    if user not in tweet_shingles:
        tweet_shingles[user] = set()
    
    tweet = final_tweets['tweet'][id]
    for i in range(len(tweet) - k-1):        
        shingle = tweet[i:i+k]
        tweet_shingles[user].add(shingle)

        if shingle not in inverted_index:
            inverted_index[shingle] = set()
        inverted_index[shingle].add((id, final_tweets.iloc[id]['user']))

In [23]:
user_similares = dict()

for user in tweet_shingles.keys():
    if user not in user_similares:
        user_similares[user] = list()
        
    for user_2 in tweet_shingles.keys():
        if user_2 not in user_similares:
            user_similares[user_2] = list()
            
        if user != user_2:
            similarity = jaccard_similarity(tweet_shingles[user], tweet_shingles[user_2])
            if 0.75 <= similarity < 1:
                if user not in user_similares[user_2]:
                    user_similares[user].append(user_2)
                    print(similarity, user, user_2)

0.9090909090909091 RuthSalinasGal KGS79
0.75 marcelotoralc jacquel87252596
0.782608695652174 Andrea25Chilena criacan


In [35]:
user_similares['RuthSalinasGal']

['KGS79']

In [25]:
final_tweets.loc[final_tweets['user'] == 'VStuckrath']

,user,tweet
7919,VStuckrath,@Jaime_Bassa Gracias por avisar para no verlo
253031,VStuckrath,@tere_marinovic Calladita no más por que si h...
634781,VStuckrath,@ChunchoXelMundo @ElisaLoncon Yo ya no se que ...
793219,VStuckrath,@Sebasti45781359 @cata_1933 @cmonckeberg Llama...
953858,VStuckrath,@RenSHevia Bienvenido al #RechazoCrece #Rechaz...
994923,VStuckrath,@Alejand64086478 @paulylm @labeasanchez Todo e...
1022602,VStuckrath,@RafaelTorrebl18 @Edoandrew4 @convencioncl @fe...
1064632,VStuckrath,@panchozen @tere_marinovic Subirán aún más lo ...
1286187,VStuckrath,@BRebolledoD17 @bobbybo73 @patricionavia @tere...
1415546,VStuckrath,@tere_marinovic @cretton15 Bien ahí... sigue l...


In [26]:
final_tweets.loc[final_tweets['user'] == 'Lore1415A']

,user,tweet
7931,Lore1415A,@Jaime_Bassa Gracias por avisar para no verlo.
41439,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
42325,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
98558,Lore1415A,@majive3105 @AmpueroAdriana Hablas de mi gener...
156524,Lore1415A,"@majive3105 @Skobar1961 @AmpueroAdriana Y si, ..."
176871,Lore1415A,@IgnacioAchurra No ha pedido disculpas pública...
216360,Lore1415A,@yasnaguzman @Alexisgomeza21 @majive3105 @Ampu...
217594,Lore1415A,@majive3105 @yasnaguzman @Alexisgomeza21 @Ampu...
260870,Lore1415A,@tere_marinovic Vamos por el rechazo de salida...
274796,Lore1415A,@majive3105 @Skobar1961 @AmpueroAdriana La mis...


In [27]:
# tweet_shingles = dict()
# inverted_index = dict()
# k=5

# for id in range(10000): 
  
#     tweet_shingles[id] = set()
    
#     for i in range(len(final_tweets['tweet'][id]) - k-1):        
#         shingle = final_tweets['tweet'][id][i:i+k]
#         tweet_shingles[id].add(shingle)

#         if shingle not in inverted_index:
#             inverted_index[shingle] = set()
#         inverted_index[shingle].add((id, final_tweets.iloc[id]['user']))

#     tweet_shingles[id] = list(tweet_shingles[id])

In [28]:
# jaccard_user = dict()

# for id in tweet_shingles.keys():
#     user_1 = final_tweets.iloc[id]['user']
    
#     for id_2 in tweet_shingles.keys():
#         user_2 = final_tweets.iloc[id_2]['user']
        
#         if user_1 != user_2:
#             shingle_similarity = jaccard_similarity(tweet_shingles[id], tweet_shingles[id_2])
            
#             if 0.8 <= shingle_similarity < 1:
                
#                 if user_1 not in jaccard_user:
#                     jaccard_user[user_1] = dict()
                    
#                 if user_2 not in jaccard_user[user_1]:
#                     jaccard_user[user_1][user_2] = [0, 0]
                    
#                 jaccard_user[user_1][user_2][1] += 1
#                 jaccard_user[user_1][user_2][0] += shingle_similarity

In [29]:
filtered_index = dict()

for k, v in inverted_index.items():
    if len(v) > 1:
        filtered_index[k] = v
        
filtered_index

{'erano': {(0, 'simonlodijo'), (580, 'don_pernil2')},
 ' @rui': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 '@ruid': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'ruido': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'uidos': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio'),
  (294, 'cas3500')},
 'idosa': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'dosaf': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'osafe': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'safes': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},
 'afest': {(0, 'simonlodijo'),
  (4, 'piaelizabethvm'),
  (18, 'galuchenta'),
  (19, 'diegosporzio')},

In [30]:
# file = "shingles.json"

# with open(file, "w") as file:
#     json.dump(tweet_shingles, file)

In [31]:
# filtered_inverted_index = dict()

# for k, v in inverted_index.items():
#     if len(v) > 1 and not list(v) in filtered_inverted_index.values():
#         filtered_inverted_index[k] = list(v)

# file = "filtered_inverted_index.json"

# with open(file, "w") as file:
#     json.dump(filtered_inverted_index, file)

In [32]:
# file = "shingles.json"

# with open(file, "r") as file:
#     tweet_shingles = json.load(file)

In [33]:
# file = "filtered_inverted_index.json"

# with open(file, "r") as file:
#     filtered_inverted_index = json.load(file)

In [34]:
# similar = []
# s = 0.4

# for pair in filtered_inverted_index.values():
#     if len(pair) == 2:
#         tweet1 = tweet_shingles[str(pair[0])]
#         tweet2 = tweet_shingles[str(pair[1])]

#         sim = jaccard_similarity(tweet1, tweet2)
#         if sim > s:
#             similar.append((pair[0], pair[1]))